In [1]:
import pandas as pd
import gc
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
import transformers
from sklearn.model_selection import train_test_split
import numpy as np
from transformers import TFAutoModel
import warnings
warnings.filterwarnings("ignore")



F:\anaconda3\envs\tf\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
F:\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
F:\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
print(gpus)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
train_df=pd.read_csv('small_data.tsv',sep='\t')
train_y=train_df['label']
train_x,val_x,train_y,val_y=train_test_split(train_df['text'],train_y,test_size=0.2,random_state=50)

AUTO = tf.data.experimental.AUTOTUNE
# Configuration of hyperparameters
EPOCHS = 3
#batch size denotes the partitioning amongst the cluster replicas.
BATCH_SIZE = 128
MAX_LEN = 192

strategy = tf.distribute.get_strategy()
print("Replicas num: ", strategy.num_replicas_in_sync)


maxlen=512
def quick_encode(texts,tokenizer, maxlen=maxlen):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        return_attention_mask=False,
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen,
        truncation=True,
    )
    return np.array(enc_di["input_ids"])

def built_model(transformer,train_data,val_data,batch_size,img_name, max_len=512):
    inp_words_ids = Input(shape =(max_len,),dtype = tf.int32,name="input_word_ids")
    seq_output = transformer(inp_words_ids)[0]
    cls_token = seq_output[:,0,:]
    output =  Dense(3,activation='softmax')(cls_token)
    model = Model(inputs =inp_words_ids,outputs=output)
    model.compile(Adam(lr=1e-5),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    model.summary()
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath='./model',
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    plot_model(
        model,to_file=img_name,
        show_shapes=True,
        show_layer_names=True,
        rankdir="TB",
        expand_nested=False,
        dpi=96)
    model.fit(
        train_data,
        steps_per_epoch=batch_size,
        validation_data=val_data,
        epochs=EPOCHS,
        callbacks=[model_checkpoint_callback]
        )
    return model

def del_obects(*args):
    for arg in args:
        del arg
        gc.collect()


with strategy.scope():
    distilbert_tokenizer = transformers.DistilBertTokenizer.from_pretrained('albert-base-v2')
    train_x_enc = quick_encode(train_x.astype(str), distilbert_tokenizer, maxlen=MAX_LEN)
    val_x_enc = quick_encode(val_x.astype(str), distilbert_tokenizer, maxlen=MAX_LEN)
    train_dataset = (
        tf.data.Dataset.from_tensor_slices((train_x_enc, train_y))
            .repeat()
            .shuffle(2048)
            .batch(BATCH_SIZE)
            .prefetch(AUTO)
    )
    valid_dataset = (
        tf.data.Dataset.from_tensor_slices((val_x_enc, val_y))
            .batch(BATCH_SIZE)
            .cache()
            .prefetch(AUTO)
    )
    transformer_layer = TFAutoModel.from_pretrained('albert-base-v2')
    model=built_model(transformer_layer, train_dataset, valid_dataset, train_x_enc.shape[0],
                "Distilbert_Multilingual_Transformer.png", max_len=MAX_LEN)
    del_obects(train_x_enc, val_x_enc, train_dataset, valid_dataset, distilbert_tokenizer, transformer_layer)

In [4]:
from DataCleaner import DataCleaner
from transformers import TFAutoModel, AlbertTokenizer

In [6]:
max_len=192
inp_words_ids = Input(shape=(max_len,), dtype=tf.int32)
transformer = TFAutoModel.from_pretrained('./al_mo', local_files_only=True)



All model checkpoint layers were used when initializing TFAlbertModel.

All the layers of TFAlbertModel were initialized from the model checkpoint at ./al_mo.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


In [7]:
transformer(inp_words_ids)

TFBaseModelOutputWithPooling(last_hidden_state=<tf.Tensor 'tf_albert_model_1/albert/encoder/albert_layer_groups_._0/albert_layers_._0/full_layer_layer_norm/batchnorm_11/add_1:0' shape=(None, 192, 768) dtype=float32>, pooler_output=<tf.Tensor 'tf_albert_model_1/albert/pooler/Tanh:0' shape=(None, 768) dtype=float32>, hidden_states=None, attentions=None)

In [9]:
seq_output = transformer(inp_words_ids)[0]

In [10]:
seq_output

<tf.Tensor 'tf_albert_model_1/albert/encoder/albert_layer_groups_._0/albert_layers_._0/full_layer_layer_norm/batchnorm_35/add_1:0' shape=(None, 192, 768) dtype=float32>

In [ ]:
cls_token = seq_output[:, 0, :]
output = Dense(2, activation='softmax')(cls_token)
model = Model(inputs=inp_words_ids, outputs=output)
model.load_weights('./albert20_192.h5')
tokenizer = AlbertTokenizer.from_pretrained('./al_to', local_files_only=True)
dataCleaner = DataCleaner()

In [102]:
train_df=pd.read_csv('datasets/gather_raw_balanced.tsv',sep='\t')
train_y=train_df['label']
train_x,val_x,train_y,val_y=train_test_split(train_df['text'],train_y,test_size=130,random_state=0)

In [103]:
val_y = pd.Series(train_df['label'].values)
val_x= pd.Series(train_df['text'].values)

In [104]:
val_y

0      1
1      1
2      1
3      1
4      1
      ..
252    0
253    0
254    1
255    1
256    1
Length: 257, dtype: int64

In [105]:
enc = tokenizer.batch_encode_plus(
            val_x,
            return_attention_mask=False,
            return_token_type_ids=False,
            pad_to_max_length=True,
            max_length=max_len,
            truncation=True,
        )
encoded=np.array(enc["input_ids"])

array([[    2,   753,  2015, ...,     0,     0,     0],
       [    2, 13024,    13, ...,     0,     0,     0],
       [    2,    13, 27803, ...,     0,     0,     0],
       ...,
       [    2,  6063,   150, ...,     0,     0,     0],
       [    2,  3535,   225, ...,     0,     0,     0],
       [    2,  3535,   225, ...,     0,     0,     0]])

In [68]:
from tensorflow.keras.metrics import Recall,Precision

In [69]:
model.compile(Adam(lr=1e-5),loss='sparse_categorical_crossentropy',metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [106]:
y_predict=model.predict(encoded)

In [71]:
y_predict[0][0]

0.45803148

In [107]:
result=[]
for item in y_predict:
    if item[0]>item[1]:
        result.append(0)
    else:
        result.append(1)

In [108]:
from sklearn import metrics
print('准确率：', metrics.accuracy_score(val_y, result))
print('分类报告:', metrics.classification_report(val_y, result))

准确率： 0.7976653696498055
分类报告:               precision    recall  f1-score   support

           0       0.72      0.86      0.79       110
           1       0.88      0.75      0.81       147

    accuracy                           0.80       257
   macro avg       0.80      0.81      0.80       257
weighted avg       0.81      0.80      0.80       257



In [1]:
import tensorflow as tf

In [2]:
x=[[[[1,2],[],[2]],[[1,2],[],[2]]],[[[1,2],[],[2]],[[1,2],[],[2]]],[[[1,2],[],[2]],[[1,2],[],[2]]]]
y=tf.convert_to_tensor([1,0,1],dtype=tf.int32)
tf.data.Dataset.from_tensor_slices((x, y))

ValueError: Can't convert non-rectangular Python sequence to Tensor.

In [92]:
a=["Fix typo doc beforethis"]
aa=tokenizer.batch_encode_plus(
            a,
            return_attention_mask=False,
            return_token_type_ids=False,
            pad_to_max_length=True,
            max_length=max_len,
            truncation=True,
        )
aaa=np.array(aa["input_ids"])

In [93]:
aaa

array([[    2,  6098, 22550,  9765,   115,  1565,     3,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [94]:
b=model.predict(aaa)
b

array([[0.897593  , 0.10240701]], dtype=float32)